<a href="https://colab.research.google.com/github/fpcaforio/grace/blob/master/CNN%2BGrad_CAM%2BNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install -U -q PyDrive
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import numpy as np
import pickle
from sklearn.cluster import KMeans
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
import time

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
#CONFIG

dataset = 'KDD-CUP99'
N_CLASSES = 2
n_clusters = 1000
n_neighbors = 2

if dataset == 'NLS-KDD':
  print('Dataset: NLS-KDD')
  pathHeatmap = '/content/drive/MyDrive/Grace/NLS-KDD/heatmap/'
  pathMagneto = '/content/drive/MyDrive/Grace/NLS-KDD/magneto/'
  size = 12
elif dataset == 'UNSW-NB15':
  print('Dataset: UNSW-NB15')
  pathHeatmap = '/content/drive/MyDrive/Grace/UNSW-NB15/heatmap/'
  pathMagneto = '/content/drive/MyDrive/Grace/UNSW-NB15/magneto/'
  size = 15
elif dataset == 'KDD-CUP99':  
  print('KDD-CUP99')
  pathHeatmap = '/content/drive/MyDrive/Grace/KDD-CUP99/heatmap/'
  pathMagneto = '/content/drive/MyDrive/Grace/KDD-CUP99/magneto/'
  size = 13

KDD-CUP99


In [5]:
f_myfile = open(pathHeatmap + 'XTrain_heatmap.pickle', 'rb')
XTrain = pickle.load(f_myfile)
f_myfile.close()

f_myfile = open(pathMagneto + 'Ytrain.pickle', 'rb')
YTrain = pickle.load(f_myfile)
f_myfile.close()

f_myfile = open(pathHeatmap + 'XTest_heatmap.pickle', 'rb')
XTest = pickle.load(f_myfile)
f_myfile.close()

f_myfile = open(pathMagneto + 'Ytest.pickle', 'rb')
YTest = pickle.load(f_myfile)
f_myfile.close()

XTrain = np.array(XTrain)
XTrain = np.uint8(255 * XTrain)
XTrain = XTrain / 255
XTrain = XTrain.reshape(-1, (size*size))

XTest = np.array(XTest)
XTest = np.uint8(255 * XTest)
XTest = XTest / 255
XTest = XTest.reshape(-1, (size*size))

print('Training Set: ', XTrain.shape)
print('Training Set: ', XTest.shape)

Training Set:  (494021, 169)
Training Set:  (311029, 169)


In [6]:
#KNeighborsClassifier

classifier = KNeighborsClassifier(n_neighbors=n_neighbors)
classifier.fit(XTrain, YTrain)
y_pred = classifier.predict(XTest)
con_mat = confusion_matrix(YTest, y_pred)

tp = con_mat[0][0]  # attacks true
fn = con_mat[0][1]  # attacs predict normal
fp = con_mat[1][0]  # normal predict attacks
tn = con_mat[1][1]  # normal as normal
attacks = tp + fn
normals = fp + tn
OA = (tp + tn) / (attacks + normals)
AA = ((tp / attacks) + (tn / normals)) / N_CLASSES
P = tp / (tp + fp)
R = tp / (tp + fn)
F1 = 2 * ((P * R) / (P + R))
FAR = fp / (fp + tn)
TPR = tp / (tp + fn)
r = (tp, fn, fp, tn, OA, AA, P, R, F1, FAR, TPR)

print('tp, fn, fp, tn, OA, AA, P, R, F1, FAR, TPR')
print(r)

tp, fn, fp, tn, OA, AA, P, R, F1, FAR, TPR
(231486, 18950, 446, 60147, 0.9376392555035061, 0.95848568942331, 0.9980770225755825, 0.9243319650529477, 0.9597900358232718, 0.0073605862063274635, 0.9243319650529477)
